In [ ]:
import requests
from datetime import timedelta, date, datetime
import json
import pandas as pd
import os
import csv
import sys
import time

In [ ]:
sys.path.append('../../code')  # Replace '/path/to/code' with the actual path to your 'code' directory
from scrape_new import daterange, get_daily_game_ids

In [ ]:
def extract_team_batting_stats(team_data, team_type, writer, custom_game_id):
    team_stats = team_data.get(team_type, {}).get('statistics', {}).get('hitting', {})
    team_stats_fielding = team_data.get(team_type, {}).get('statistics', {}).get('fielding', {})

    if team_stats:
        # Extract batting and fielding stats
        stat = {
            'home_away': team_type,
            'ab': team_stats['overall'].get('ab', 'N/A'),
            'r': team_stats['overall']['runs'].get('total', 'N/A'),
            'h': team_stats['overall']['onbase'].get('h', 'N/A'),
            'rbi': team_stats['overall'].get('rbi', 'N/A'),
            'bb': team_stats['overall']['onbase'].get('bb', 'N/A'),
            'so': team_stats['overall']['outs'].get('ktotal', 'N/A'),
            'pa': team_stats['overall'].get('ap', 'N/A'),
            'batting_avg': team_stats['overall'].get('avg', 'N/A'),
            'onbase_perc': team_stats['overall'].get('obp', 'N/A'),
            'slugging_perc': team_stats['overall'].get('slg', 'N/A'),
            'onbase_plus_slugging': team_stats['overall'].get('ops', 'N/A'),
            'pitches': team_stats['overall'].get('pitch_count', 'N/A'),
            'strikes_total': team_stats['overall'].get('pitches', {}).get('ktotal', 'N/A'),
            'wpa_bat': 'N/A',
            'leverage_index_avg': 'N/A',
            'wpa_bat_pos': 'N/A',
            'wpa_bat_neg': 'N/A',
            're24_bat': 'N/A',
            'po': team_stats_fielding['overall'].get('po', 'N/A'),
            'a': team_stats_fielding['overall'].get('a', 'N/A'),
            'game_id': custom_game_id
        }
        writer.writerow(stat)

In [ ]:
def fetch_and_store_batting_stats(api_key, start_date, delay_seconds=1):
    file_path = '../../data/batting-test-3.csv'
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    fieldnames = [
        'home_away', 'ab', 'r', 'h', 'rbi', 'bb', 'so', 'pa',
        'batting_avg', 'onbase_perc', 'slugging_perc', 'onbase_plus_slugging',
        'pitches', 'strikes_total', 'wpa_bat', 'leverage_index_avg',
        'wpa_bat_pos', 'wpa_bat_neg', 're24_bat', 'po', 'a', 'game_id'
    ]

    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if os.stat(file_path).st_size == 0:
            writer.writeheader()

        for current_date in daterange(start_date):
            year = current_date.year
            month = current_date.month
            day = current_date.day
            game_ids = get_daily_game_ids(api_key, year, month, day)
            for game_id in game_ids:  # Limiting to the first game for testing
                url = f'https://api.sportradar.com/mlb/trial/v7/en/games/{game_id}/extended_summary.json?api_key={api_key}'
                response = requests.get(url)

                if response.status_code == 200:
                    data = response.json()
                    game_info = data['game']
                    date = datetime.strptime(game_info['scheduled'], "%Y-%m-%dT%H:%M:%S%z").strftime("%Y%m%d")
                    day_night = game_info['day_night']
                    custom_game_id = f"{game_info['away']['abbr']}{game_info['home']['abbr']}{date}{day_night}"

                    # Directly write team stats for home and away to CSV
                    extract_team_batting_stats(data['game'], 'home', writer, custom_game_id)
                    extract_team_batting_stats(data['game'], 'away', writer, custom_game_id)

                    # Manage API rate limits
                    time.sleep(delay_seconds)
                else:
                    print(f"Failed to fetch data: {response.status_code} {response.text}")
                    time.sleep(delay_seconds)

    print(f"Team batting data written to '{file_path}'")

In [ ]:
api_key = ''  # Replace with your actual API key
start_date = datetime(2024, 3, 1).date()
fetch_and_store_batting_stats(api_key, start_date, delay_seconds=1)